In [3]:
import os

# Paleo modeling


# Model setup

Simulations are run using the COAWST implementation of ROMS (version VERSION). The horizontal grid has been used in several studies (Hetland and Campbell, 2007; Zhang et al, 2014) and output is used operationally by the Texas General Land Office and NOAA's Office of Response and Restoration.

## Boundaries
All boundaries are closed. Walls are no-slip and bottom friction is used with (WHAT VALUES). There is no meteorological forcing.

## Bathymetry

The sea level was about 120 meters lower 19,000 years ago at the end of the last glacial period and the Mississippi river entered the Gulf at the Mississippi canyon.

![](images/bathy.png)

### Shifting
In the script `setup/bathy/bathy_shift.py`, I subtract 120 meters from all of the depths, mask out anything that went negative since it is now land, make everything have a minimum depth of 25 meters, fix a couple of weird holes in the mask by hand (not sure these ended up being important to model stability), and then update the masks for all grids. The output of this is `setup/bathy/grid-shifted.nc`.

### Smoothing
The script `setup/bathy/bathy_smooth.py` takes in `grid-shifted.nc` and smoothes the bathymetry to a Beckman/Haidvogel number of 0.4. It's hard to get the code installed, but I had it working on my old laptop and found that the different algorithms for smoothing tend to give fairly similar results, so I am just using "Positive" now.

### Vertical grid
The number of vertical levels is 20 with Vtransform=2 and Vstretching=4 to achieve a Haney (CITE) metric of 7.8.

## T and S initialization

### Data
The simulations are initialized with a uniform density gradient (with depth) calculated using the linear equation of state with temperature and salinity averaged over the profiles available in the World Ocean Atlas (1955-2012) from the deep Gulf of Mexico (Station number 28569 (B): 93.5W, 24.5N). The ODV [WOA 2013 data](https://odv.awi.de/en/data/ocean/world_ocean_atlas_2013/
) ([data link](https://odv.awi.de/fileadmin/user_upload/odv/data/WOA13/WOA13_1.00deg_1955-2012_Annual.zip
)) is saved into file `setup/initialization/data_from_WOA13_1.00deg_1955-2012_Annual_centerGOM.nc` and is shown here:

<img src="images/Profiles_AnalyzedValues_centerGOM.gif" width="70%">

### Present day background values
The background salinity and temperature are given by values from the bottom of the WOA profile where the values are more consistent: $S_0=35.0$ g kg$^{-1}$, and $T_0=4.2$ $^\circ$C. These are calculated from data arrays in `setup/initialization/initialization.py`, taking the bottom 20 bins (1950 to 3800 depth). Then I used a basic [T-S diagram](http://glossary.periodni.com/download_image.php?name=ts-diagram.png&source=T-S+diagram
) to read off the background density of $\rho_0=1027.8$ kg/m$^3$:

<img src="images/ts.png" alt="ts" width="30%">

### Paleo values
Based on Niall's work, the paleo temperature profile is then uniformly 2 degrees cooler and 3.5% more saline across whole water column. The background numbers are then $S_0=36.225$ g kg$^{-1}$, and $T_0=2.2$ $^\circ$C. I read the background density off the T-S diagram: $\rho_0=1028.9$ kg/m$^3$.

### Linear equation of state
I used the linear equation of state to calculate the paleo density profile, using the paleo background numbers. The linear equation of state is given by the equation below where $S_{COEF}=7.6\times 10^{-4}$ (g kg$^{-1}$)$^{-1}$ and $T_{COEF}=1.7\times 10^{-4}$ $^\circ$C$^{-1}$ from the ROMS model. 
$$ \rho = \rho_0 *\left[1.0 + S_{COEF} (S-S_0) - T_{COEF} (T-T_0)\right] $$

The resultant salinity and temperature profiles are on the left in the figure below.

In [12]:
os.chdir('../setup/initialization')
os.system('python make_initialization.py True False');
os.chdir('../../notebook')

![profiles](../setup/initialization/WOA_profiles.png)

In case the figures aren't showing above, here is the image:

![](../setup/initialization/WOA_profiles.png)

### Density setup for simulation

After finding the density profile for the paleo initialization, I use the linear equation of state to calculate the equivalent temperature profile that would make that density profile if the salinity is uniformly equal to the background salinity. That is, $S=S_0$, so $ \rho = \rho_0 *\left[1.0 - T_{COEF} (T-T_0)\right] $ and 
$$ T = -\frac{\rho}{\rho_0 T_{COEF}} - T_0 $$

This is a numerical convenience so that salinity can be used as a tracer representing the input river water. The paleo density profile and temperature profile that gives the same density profile with no salinity changes are shown on the right above.

### Initialization file
The initialization file `ocean_ini.nc` is created by `python make_initialization.py False True`.

## River input

### Locations and structure

I have the river coming in around the Mississippi canyon (see bathymetry above). The river location options are visualized in the figures below, as made in `setup/river/river_locations.py`. Since the locations are defined on the u and v grid by the cell wall that the river water is actually exiting through, the proper grid labels are shown for each of the 3 main numerical grids.

<img src="images/masku.png" alt="ts" width="50%">
<img src="images/maskv.png" alt="ts" width="50%">

The river forcing file, `tglo_river_frc_2010-01-01.nc`, is made in `setup/river/river_frc_tglo.py`. This file is referenced in time to 'days since 1970-01-01 00:00:00' and starts on 2010-01-01, though the date isn't relevant for this simulation except to keep track of time passing. The time length is controlled in this file (currently a year of 30 day months) and the output is every 3 hours. The river is input over the full 20 vertical layers. The amount of river water in each layer is weighted by the thickness of the layer so that the profile is uniform over the water column. There are 24 input locations that the river water is divided across at each time step, and the amount out each cell face is weight by the surface area of the cell wall.

### Amount
The total volumne of water being input through the river is $1e14$ m$^3$, which is from Clarke et al 2004’s flood volume of 151,000 km^3 over 0.5 year. I used a Gaussian to represent the flood pulse:

$$Q(t) = A e^\left( - \frac{t-mid^2}{2w^2}\right)$$

where  $Q(t)$ is the flux in m^3/s, $A$ is the amplitude [m$^3$/s], $mid$ is the middle point of the gaussian, and $w$ is the width (0.15 * the 12 month runtime). I solve for A:

$$ \Sigma_t Q(t) = A * \Sigma_t e^{()} $$

and $\Sigma_t Q(t)$ is the volume, so $A = V / \Sigma_t e^{()} * dt  = 8561003 m^3/s$ (calculated in `setup/river/river_frc_tglo.py`).  Then I can solve for Gaussian.

<img src="images/river_pulse.png" alt="ts" width="20%">

### CFL and time step

The velocity for this flow rate is:

$$ u = Q_{max} / (\Delta x * \Delta z) = 460977 m^3/s / (5000 m * 25 m) = 3.7 m/s $$

The CFL condition is satisfied for

$$ t < 0.5 ∆x/u = 0.5 * 5000m / 3.7 m/s = 675 s $$

though the actual time step being used ends up being smaller.

### Density

The density of the river water is controlled by the salinity and the temperature is set to the background temperature. I choose the desired density of the river water and use the linear equation of state to solve for the necessary salinity.

## Key input parameters

* NTIMES == 86400  # 1 year simulation time
* DT == 360.0d0  # 6 min time step
* NHIS == 240  # daily output
* NDEFHIS == 86400  # all output in 1 year-long file  


# Initial results

I ran a series of four simulations with a range of river densities to span from the top to bottom of the water column. 

## low density, top of the water column

`new1`, river salinity = 33.9 (density 1026) (background salinity 36.225)
<img src="images/new1-180.png" alt="ts" width="30%">
<img src="images/new1-360.png" alt="ts" width="30%">

## medium-low density

`new2`, river salinity = 35.2 (density 1027)
<img src="images/new2-180.png" alt="ts" width="30%">
<img src="images/new2-360.png" alt="ts" width="30%">

## medium-high density

`new3`, river salinity = 36.5 (density 1028)
<img src="images/new3-180.png" alt="ts" width="30%">
<img src="images/new3-360.png" alt="ts" width="30%">

## high density, bottom of the water column

`new4`, river salinity = 37.8 (density 1029)
<img src="images/new4-180.png" alt="ts" width="30%">
<img src="images/new4-360.png" alt="ts" width="30%">


# Possible next directions

Several sources show a sea level rise 19,000 years ago. Clark et al (2004) the rise was at least 10 meters. Yokoyama et al (2000) say a rapid rise of 10-15 meters. Clark et al (20xx) the rise was at least 10 meters. Somebody was 5 meters (Mortlock?). So we are planning to use between a 5 and 10 meter sea level rise.

Given a surface area of the ocean of $3.61e8$ (from the way Mortlock calculated this for his work?), this gives a volume of water range of 1.805 to 3.61 million km$^3$ of seawater input into the ocean. (This is compared with the volume of river water I have previously run with: 100,000 km^3).

Niall has evidence in his cores that these inputs occurred over three different events, with time in between.

The time range is unknown because the dating methods are not precise enough. However, we do know that the range must be less than 500 years because of WHO'S data in WHAT PAPER. At least paper (WHICH?) also indicated a flood of size 10,000 km$^3$/year (IS THAT CORRECT?).

For reference, the typical present day Mississippi discharge per year is (from Wikipedia's mean number):

$$ 16792 m^3/s *(365 days/year) * 24 hours/day * 3600 seconds/hour) / 1000(m/km)^3 \approx 500 km^3/year$$

We probably only need to run one of these events. We could run simulations for several time lengths, with different approaches:

1. For a short, strong flood, I would need to use a very small time step but the simulation would be shorter.
1. For a long, weak flood, I could use a bigger time step and run a few targeted years during the long simulation time period since the time scale of the flood would be so long relative to the dynamics of the water being input into the Gulf.